In [5]:
import django_initializer
from ontrack.market.models.participant import ParticipantStatsActivity, ParticipantActivity
import pandas as pd
import numpy as np
from ontrack.utils.datetime import DateTimeHelper as dt

In [6]:
def color_boolean(val):
    color =''
    foreground = ''
    if val == "BEARISH":
        color = 'red'
        foreground = 'black'        
    elif val == "BULLISH":
        color = 'green'        
        foreground = 'black'  
    return f'background-color: {color}; color: {foreground}'

In [24]:
d = dt.get_past_date(days=10)
participants = ParticipantStatsActivity.backend.filter(date__gte=d).order_by("date")
df = pd.DataFrame(list(participants.values("date", "client_type", "instrument", "no_of_contracts_bought", "value_of_contracts_bought", "value_of_contracts_sold", "no_of_contracts_sold", "open_interest")))
common_names = {
    "client_type": "type",
    "instrument": "instrument",
    "no_of_contracts_bought": "bought",    
    "value_of_contracts_bought": "bought_value",
    "no_of_contracts_sold": "sold",
    "value_of_contracts_sold": "sold_value",
    "open_interest": "oi"
}
# df.set_index("date", inplace = True)
df.rename(columns=common_names, errors="ignore", inplace=True)
df['balance_contracts'] = (df["bought"] - df["sold"])
df['balance_value'] = (df["bought_value"] - df["sold_value"])
df['net_contracts'] = (df.groupby(['type', 'instrument'])['balance_contracts'].transform(lambda x: x.diff()))
df['net_value'] = (df.groupby(['type', 'instrument'])['balance_value'].transform(lambda x: x.diff()))
df['change_oi'] = (df.groupby(['type', 'instrument'])['oi'].transform(lambda x: x.diff()))
df['pcr_contracts'] = (df["bought"] / (df["bought"] + df["sold"])).astype(float)

conditions = [
    (df["change_oi"] > 0) & (df['balance_contracts'] > 0),
    (df["change_oi"] > 0) & (df['balance_contracts'] < 0),
    (df["change_oi"] < 0) & (df['balance_contracts'] > 0),
    (df["change_oi"] < 0) & (df['balance_contracts'] < 0)]
choices = ["LONG BUILDUP", "SHORT BUILDUP", "SHORT COVERING", "LONG UNWINDING"]
df["position"] = np.select(conditions, choices)

choices = ["BULLISH", "BEARISH", "BULLISH", "BEARISH"]
df["direction"] = np.select(conditions, choices)

df_f = df.tail(4)
df_s = df_f.style.applymap(color_boolean, subset=['direction'])

df_s

,date,type,instrument,bought,bought_value,sold_value,sold,oi,balance_contracts,balance_value,net_contracts,net_value,change_oi,pcr_contracts,position,direction
28,2022-12-09,FII,FUTIDX,26676.0000,2687.9400,3667.5100,36557.0000,175942.0000,-9881.0000,-979.5700,-17052.0000,-1749.4300,-3423.0000,0.421868,LONG UNWINDING,BEARISH
29,2022-12-09,FII,OPTIDX,8546633.0000,880845.2100,877672.0400,8517633.0000,2064757.0000,29000.0000,3173.1700,-84565.0000,-7997.3100,630714.0000,0.500850,LONG BUILDUP,BULLISH
30,2022-12-09,FII,FUTSTK,128795.0000,9085.9500,11073.2000,156227.0000,1939361.0000,-27432.0000,-1987.2500,-22285.0000,-2014.5400,12846.0000,0.451877,SHORT BUILDUP,BEARISH
31,2022-12-09,FII,OPTSTK,159347.0000,11647.7200,11886.7400,162878.0000,150979.0000,-3531.0000,-239.0200,-2050.0000,-97.3300,6727.0000,0.494521,SHORT BUILDUP,BEARISH


In [25]:
df = df[df["instrument"]=="FUTIDX"]
df = df[["date", "balance_value", "position", "direction", "instrument"]]

df_s = df.style.applymap(color_boolean, subset=['direction'])
df_s

,date,balance_value,position,direction,instrument
0,2022-11-30,1109.0300,0,0,FUTIDX
4,2022-12-01,-526.0900,LONG UNWINDING,BEARISH,FUTIDX
8,2022-12-02,-3917.8300,LONG UNWINDING,BEARISH,FUTIDX
12,2022-12-05,239.6600,SHORT COVERING,BULLISH,FUTIDX
16,2022-12-06,-1533.4200,SHORT BUILDUP,BEARISH,FUTIDX
20,2022-12-07,-1218.4100,SHORT BUILDUP,BEARISH,FUTIDX
24,2022-12-08,769.8600,LONG BUILDUP,BULLISH,FUTIDX
28,2022-12-09,-979.5700,LONG UNWINDING,BEARISH,FUTIDX


In [8]:
d = dt.get_past_date(days=5)
instruments = ["FUTIDX", "OPTIDX", "CASH"]
client_types = ["CLIENT", "PRO", "DII", "FII"]

# instruments = ["FUTIDX",]
# client_types = ["FII", ]
participants = ParticipantActivity.backend.filter(date__gte=d, instrument__in=instruments, client_type__in=client_types).order_by("date", "client_type", "instrument")
df = pd.DataFrame(list(participants.values("date", "client_type", "instrument", "option_type", "buy_amount", "sell_amount", "net_amount")))
common_names = {
    "instrument": "instrument"
}

# # df.set_index("date", inplace = True)
df.rename(columns=common_names, errors="ignore", inplace=True)
# df['balance'] = df["bought"] - df["sold"] 
df['long'] = (df.groupby(['client_type', 'instrument', "option_type"], dropna=False)['buy_amount'].transform(lambda x: x.diff()))
df['short'] = (df.groupby(['client_type', 'instrument', "option_type"], dropna=False)['sell_amount'].transform(lambda x: x.diff()))
df["net"] = df['long'] - df['short']

a = np.array(df['buy_amount'], dtype=float)
b = np.array(df['sell_amount'], dtype=float)
df["lsr"] = np.divide(a, b, out=np.zeros_like(a), where= b!=0)
df['lsr'] = np.round(df['lsr'], 2)

a = np.array(df['buy_amount'], dtype=float)
b = np.array((df['buy_amount'] + df['sell_amount']), dtype=float)
df['pcr'] = np.divide(a, b, out=np.zeros_like(a), where= b!=0)
df['pcr'] = np.round(df['pcr'], 2)

conditions = [
    (df['option_type'] != "PE" ) & (df['long'] > df['short']) & (df['long'] + df['short'] > 0),
    (df['option_type'] != "PE" ) & (df['long'] < df['short']) & (df['long'] + df['short'] > 0),
    (df['option_type'] != "PE" ) & (df['long'] < df['short']) & (df['long'] + df['short'] < 0),
    (df['option_type'] != "PE" ) & (df['long'] > df['short']) & (df['long'] + df['short'] < 0),
    
    (df['option_type'] == "PE" ) & (df['long'] > df['short']) & (df['long'] + df['short'] > 0),
    (df['option_type'] == "PE" ) & (df['long'] < df['short']) & (df['long'] + df['short'] > 0),
    (df['option_type'] == "PE" ) & (df['long'] < df['short']) & (df['long'] + df['short'] < 0),
    (df['option_type'] == "PE" ) & (df['long'] > df['short']) & (df['long'] + df['short'] < 0)]
choices = [
    "LONG BUILDUP", 
    "SHORT BUILDUP", 
    "LONG UNWINDING", 
    "SHORT COVERING", 
    "SHORT BUILDUP", 
    "LONG BUILDUP", 
    "SHORT COVERING", 
    "LONG UNWINDING", ]
df["position"] = np.select(conditions, choices, default="NA")

choices = ["BULLISH", "BEARISH", "BEARISH", "BULLISH", "BEARISH", "BULLISH" , "BULLISH" , "BEARISH"]
df["direction"] = np.select(conditions, choices, default="NA")

df = df.tail(14)
df = df.style.applymap(color_boolean, subset=['direction'])

df

,date,client_type,instrument,option_type,buy_amount,sell_amount,net_amount,long,short,net,lsr,pcr,position,direction
56,2022-12-09,CLIENT,FUTIDX,None,224164.0000,192083.0000,32081.0000,1565.0000,-762.0000,2327.0000,1.170000,0.540000,LONG BUILDUP,BULLISH
57,2022-12-09,CLIENT,OPTIDX,PE,2349362.0000,2691654.0000,-342292.0000,458780.0000,628043.0000,-169263.0000,0.870000,0.470000,LONG BUILDUP,BULLISH
58,2022-12-09,CLIENT,OPTIDX,CE,3226996.0000,3204104.0000,22892.0000,1515423.0000,1329186.0000,186237.0000,1.010000,0.500000,LONG BUILDUP,BULLISH
59,2022-12-09,DII,CASH,None,6555.2500,6053.6200,501.6300,258.2100,528.8700,-270.6600,1.080000,0.520000,SHORT BUILDUP,BEARISH
60,2022-12-09,DII,FUTIDX,None,24806.0000,67872.0000,-43066.0000,-1.0000,-3050.0000,3049.0000,0.370000,0.270000,SHORT COVERING,BULLISH
61,2022-12-09,DII,OPTIDX,CE,719.0000,0.0000,719.0000,0.0000,0.0000,0.0000,0.000000,1.000000,NA,NA
62,2022-12-09,DII,OPTIDX,PE,47573.0000,0.0000,47573.0000,0.0000,0.0000,0.0000,0.000000,1.000000,NA,NA
63,2022-12-09,FII,CASH,None,7515.5400,7673.5500,-158.0100,-1198.8100,-2172.4700,973.6600,0.980000,0.490000,SHORT COVERING,BULLISH
64,2022-12-09,FII,FUTIDX,None,101239.0000,74703.0000,26536.0000,-6652.0000,3229.0000,-9881.0000,1.360000,0.580000,LONG UNWINDING,BEARISH
65,2022-12-09,FII,OPTIDX,CE,608567.0000,471115.0000,137452.0000,170440.0000,232441.0000,-62001.0000,1.290000,0.560000,SHORT BUILDUP,BEARISH
